# Analytics of MobileNet V1 Model Sizes


In [1]:
import os, sys, math, datetime
import psutil

# import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image

# Import the necessary MLTK APIs
from mltk.core import view_model, summarize_model, profile_model
from mltk.utils.commander import query_platform

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.layers import (
    Input,
    Dense,
    Flatten,
    Conv2D,
    MaxPooling2D,
    GlobalAveragePooling2D,
    BatchNormalization,
    Concatenate,
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths

import plotly.express as px
# enable plotly in VS Studio Code
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
# Configure pandas to show all columns & rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [3]:
models_dir = initialize()

In [4]:
base_model_name = "mobilenetv1"

In [21]:
csv_file_name = f"{base_model_name}_stats.csv"
csv_path = Path.cwd().joinpath(csv_file_name)

In [50]:
df = pd.read_csv(csv_path)

In [51]:
def highlight_cells(val):
    color = 'green' if val < 220 else 'red'
    return f'background-color: {color}'

In [53]:
df["params_kb"] = df["total_params"] * 4 / 1024
df["tflite_INT8_model_size_kb"] = df["tflite_model_size_kb"] / 4
df.insert(6,"tflite_INT8_model_size_kb" , df.pop("tflite_INT8_model_size_kb"))
df.insert(14,"dropout_rate" , df.pop("dropout_rate"))
df.sort_values(["tflite_INT8_model_size_kb", "model_name"], inplace=True)
#df.head(10)
df.style.applymap(highlight_cells,
    subset=["tflite_INT8_model_size_kb"])

,img_res,classes,channels,alpha,model_name,MACs,tflite_INT8_model_size_kb,FLOPs,total_params,trainable_params,non_trainable_params,model_size_kb,tflite_model_size_kb,params_kb,dropout_rate
10,128,3,1,0.050000,mobilenetv1_0.05_128_c1_o3_000,692.249,14.736328,1.606,12573,11507,1066,307.199219,58.945312,49.113281,0.500000
29,96,3,1,0.050000,mobilenetv1_0.05_96_c1_o3_000,389.457 k,14.736328,903.717 k,12573,11507,1066,307.199219,58.945312,49.113281,0.500000
18,96,3,1,0.050000,mobilenetv1_0.05_96_c1_o3_000,389.457,14.736328,903.717,12573,11507,1066,307.199219,58.945312,49.113281,0.500000
33,96,3,1,0.050000,mobilenetv1_0.05_96_c1_o3_000,389.457 k,14.736328,903.717 k,12573,11507,1066,307.199219,58.945312,49.113281,0.100000
14,128,3,3,0.050000,mobilenetv1_0.05_128_c3_o3_000,765.977,14.753906,1.754,12591,11525,1066,307.199219,59.015625,49.183594,0.500000
34,96,3,3,0.050000,mobilenetv1_0.05_96_c3_o3_000,430.929 k,14.753906,986.661 k,12591,11525,1066,307.199219,59.015625,49.183594,0.100000
22,96,3,3,0.050000,mobilenetv1_0.05_96_c3_o3_000,430.929,14.753906,986.661,12591,11525,1066,307.199219,59.015625,49.183594,0.500000
9,128,3,1,0.100000,mobilenetv1_0.1_128_c1_o3_000,2.351,40.119141,5.175,41220,39052,2168,418.714844,160.476562,161.015625,0.500000
17,96,3,1,0.100000,mobilenetv1_0.1_96_c1_o3_000,1.323,40.119141,2.911,41220,39052,2168,418.714844,160.476562,161.015625,0.500000
13,128,3,3,0.100000,mobilenetv1_0.1_128_c3_o3_000,2.572,40.171875,5.618,41274,39106,2168,418.667969,160.687500,161.226562,0.500000


In [12]:
len(df)

35

In [35]:
df_c1 = df.loc[(df['channels'] == 1)]
df_c1_sorted = df_c1.sort_values(['img_res', 'tflite_INT8_model_size_kb'],
              ascending = [True, True])
df_c1_sorted

,img_res,classes,channels,alpha,dropout_rate,model_name,MACs,FLOPs,total_params,trainable_params,non_trainable_params,model_size_kb,tflite_model_size_kb,params_kb,tflite_INT8_model_size_kb
1,96,3,1,0.05,0.5,mobilenetv1_0.05_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,307.199219,58.945312,49.113281,14.736328
21,96,3,1,0.05,0.5,mobilenetv1_0.05_96_c1_o3_000,389.457,903.717,12573,11507,1066,307.199219,58.945312,49.113281,14.736328
0,96,3,1,0.10,0.5,mobilenetv1_0.1_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,418.714844,160.476562,49.113281,40.119141
20,96,3,1,0.10,0.5,mobilenetv1_0.1_96_c1_o3_000,1.323,2.911,41220,39052,2168,418.714844,160.476562,161.015625,40.119141
4,96,3,1,0.20,0.5,mobilenetv1_0.2_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,824.503906,545.207031,49.113281,136.301758
19,96,3,1,0.20,0.5,mobilenetv1_0.2_96_c1_o3_000,4.625,9.789,144942,140590,4352,824.503906,545.207031,566.179688,136.301758
18,96,3,1,0.25,0.5,mobilenetv1_0.25_96_c1_o3_000,7.158,15.011,221907,216435,5472,1124.664062,835.320312,866.824219,208.830078
6,96,3,1,0.50,0.5,mobilenetv1_0.5_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,3525.070312,3183.664062,49.113281,795.916016
7,96,3,1,1.00,0.5,mobilenetv1_1_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,12924.093750,12479.351562,49.113281,3119.837891
13,128,3,1,0.05,0.5,mobilenetv1_0.05_128_c1_o3_000,692.249,1.606,12573,11507,1066,307.199219,58.945312,49.113281,14.736328


In [8]:
df_c3 = df.loc[(df['channels'] == 3)]
df_c3_sorted = df_c3.sort_values(['img_res', 'tflite_INT8_model_size_kb'],
              ascending = [True, True])
df_c3_sorted

,img_res,classes,channels,alpha,dropout_rate,model_name,MACs,FLOPs,total_params,trainable_params,non_trainable_params,model_size_kb,tflite_model_size_kb,params_kb,tflite_INT8_model_size_kb
3,96,3,3,0.05,0.5,mobilenetv1_0.05_96_c3_o3_000,389.457 k,903.717 k,12573,11507,1066,307.199219,59.015625,49.113281,14.753906
25,96,3,3,0.05,0.5,mobilenetv1_0.05_96_c3_o3_000,430.929,986.661,12591,11525,1066,307.199219,59.015625,49.183594,14.753906
2,96,3,3,0.10,0.5,mobilenetv1_0.1_96_c3_o3_000,389.457 k,903.717 k,12573,11507,1066,418.667969,160.687500,49.113281,40.171875
24,96,3,3,0.10,0.5,mobilenetv1_0.1_96_c3_o3_000,1.447,3.16,41274,39106,2168,418.667969,160.687500,161.226562,40.171875
34,96,3,3,0.10,0.5,mobilenetv1_0.1_96_c3_o3_000,1.447 M,3.160 M,41274,39106,2168,418.667969,160.687500,161.226562,40.171875
5,96,3,3,0.20,0.5,mobilenetv1_0.2_96_c3_o3_000,389.457 k,903.717 k,12573,11507,1066,824.535156,545.628906,49.113281,136.407227
23,96,3,3,0.20,0.5,mobilenetv1_0.2_96_c3_o3_000,4.874,10.287,145050,140698,4352,824.535156,545.628906,566.601562,136.407227
22,96,3,3,0.25,0.5,mobilenetv1_0.25_96_c3_o3_000,7.49,15.675,222051,216579,5472,1124.703125,835.882812,867.386719,208.970703
8,96,3,3,0.50,0.5,mobilenetv1_0.5_96_c3_o3_000,389.457 k,903.717 k,12573,11507,1066,3525.140625,3184.789062,49.113281,796.197266
9,96,3,3,1.00,0.5,mobilenetv1_1_96_c3_o3_000,389.457 k,903.717 k,12573,11507,1066,12925.531250,12481.601562,49.113281,3120.400391


In [9]:
df_96_c1 = df.loc[(df['channels'] == 1) & (df['img_res'] == 96)]
df_96_c1

,img_res,classes,channels,alpha,dropout_rate,model_name,MACs,FLOPs,total_params,trainable_params,non_trainable_params,model_size_kb,tflite_model_size_kb,params_kb,tflite_INT8_model_size_kb
0,96,3,1,0.10,0.5,mobilenetv1_0.1_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,418.714844,160.476562,49.113281,40.119141
1,96,3,1,0.05,0.5,mobilenetv1_0.05_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,307.199219,58.945312,49.113281,14.736328
4,96,3,1,0.20,0.5,mobilenetv1_0.2_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,824.503906,545.207031,49.113281,136.301758
6,96,3,1,0.50,0.5,mobilenetv1_0.5_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,3525.070312,3183.664062,49.113281,795.916016
7,96,3,1,1.00,0.5,mobilenetv1_1_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,12924.093750,12479.351562,49.113281,3119.837891
18,96,3,1,0.25,0.5,mobilenetv1_0.25_96_c1_o3_000,7.158,15.011,221907,216435,5472,1124.664062,835.320312,866.824219,208.830078
19,96,3,1,0.20,0.5,mobilenetv1_0.2_96_c1_o3_000,4.625,9.789,144942,140590,4352,824.503906,545.207031,566.179688,136.301758
20,96,3,1,0.10,0.5,mobilenetv1_0.1_96_c1_o3_000,1.323,2.911,41220,39052,2168,418.714844,160.476562,161.015625,40.119141
21,96,3,1,0.05,0.5,mobilenetv1_0.05_96_c1_o3_000,389.457,903.717,12573,11507,1066,307.199219,58.945312,49.113281,14.736328
32,96,3,1,0.05,0.5,mobilenetv1_0.05_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,307.199219,58.945312,49.113281,14.736328


In [10]:
df_005_c1 = df.loc[(df['channels'] == 1) & (df['alpha'] == 0.05)]
df_005_c1

,img_res,classes,channels,alpha,dropout_rate,model_name,MACs,FLOPs,total_params,trainable_params,non_trainable_params,model_size_kb,tflite_model_size_kb,params_kb,tflite_INT8_model_size_kb
1,96,3,1,0.05,0.5,mobilenetv1_0.05_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,307.199219,58.945312,49.113281,14.736328
13,128,3,1,0.05,0.5,mobilenetv1_0.05_128_c1_o3_000,692.249,1.606,12573,11507,1066,307.199219,58.945312,49.113281,14.736328
21,96,3,1,0.05,0.5,mobilenetv1_0.05_96_c1_o3_000,389.457,903.717,12573,11507,1066,307.199219,58.945312,49.113281,14.736328
32,96,3,1,0.05,0.5,mobilenetv1_0.05_96_c1_o3_000,389.457 k,903.717 k,12573,11507,1066,307.199219,58.945312,49.113281,14.736328


In [11]:
fig = px.bar(df_005_c1, x="img_res", y= "tflite_INT8_model_size_kb", text_auto=".2s",
    title=f"TFLite Model Size in kB - INT8")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="MCU constraint", 
              #annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig.update_layout(hovermode="y")
fig.show()

In [11]:
fig = px.scatter(df, x= "channels", y= "tflite_INT8_model_size_kb")
fig.update_layout(hovermode="y")
# fig.update_layout(
#     hoverlabel=dict(
#         bgcolor="white",
#         font_size=16,
#         font_family="Rockwell"
#     )
#)
fig.show()